In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cooler
import lavaburst

/home/dmitry/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import TADcalling
from TADcalling.CallerClasses import *

In [3]:
lc = LavaburstCaller(['S2'], ['./data/S2.20000.cool'], 'cool', assembly='dm3', resolution=20000, balance=True, chr='chr2L')

In [5]:
lc.call(0.9)

{'S2': {0.9: <TADcalling.DataClasses.GenomicRanges at 0x7f0ff0a287b8>,
  1.9: <TADcalling.DataClasses.GenomicRanges at 0x7f0ff0a205f8>}}

In [5]:
GenomicRanges

TADcalling.DataClasses.GenomicRanges

In [6]:
lc.segmentation2df()

,bgn,end,label,params
0,0,6,S2,0.9
1,6,15,S2,0.9
2,21,27,S2,0.9
3,27,83,S2,0.9
4,84,108,S2,0.9
5,108,112,S2,0.9
6,112,131,S2,0.9
7,132,137,S2,0.9
8,137,143,S2,0.9
9,143,149,S2,0.9
